<a href="https://colab.research.google.com/github/fab2112/Q-learning-taxi-v3-demo/blob/main/Q_learning_taxi_v3_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demonstração do algorítmo Q-learning no ambiente *taxi-v3* *openai* *gym*
https://gym.openai.com/envs/Taxi-v3/

# Importa bibliotecas


In [50]:
from time import sleep, time
import numpy as np
import gym.spaces
import matplotlib.pyplot as plt
from IPython.display import clear_output
import seaborn as sns

# Define parâmetros, instancia ambiente e inicializa Q_table.

In [51]:
# Instancia ambiente Taxi-v3 no gym
env = gym.make("Taxi-v3")

# Inicializa tabela Q
Q_table = np.random.random([env.observation_space.n, env.action_space.n])

# Parâmetros Q-learning
gamma = 0.2
alpha = 0.4
epsilon = 0.2
epsilon_decay = 0.99
episodes = 20000

total_epochs = 0
total_rewards = []

# Fase de treino

In [ ]:
for episode in range(episodes):

    time_before = float(time() * 1000)

    reward = 0
    epochs = 0
    rewards_by_episode = 0
    state = env.reset()

    done = False
    while not done:

        # Seleciona uma ação aleatória - Exploration
        if np.random.rand() < epsilon:
            action = env.action_space.sample()
        # Seleciona uma ação com maior valor Q - Exploitation
        else:
            action = np.argmax(Q_table[state])

        # Move para próximo estado e atualiza recompensa
        next_state, reward, done, info = env.step(action)

        # Atualiza valor Q(s, a) - Equação de Bellman
        Q_table[state, action] = \
            Q_table[state, action] + alpha * (reward + gamma * (np.max(Q_table[next_state])) - Q_table[state, action])

        # Atualiza state para o estado atual
        state = next_state
        # Computa rewards por episodio
        rewards_by_episode += reward
        # Computa epocas por episodio
        epochs += 1

        # Render treino
        #env.render()
        #sleep(0.06)

    # Monitoramento da latência do treino por espiósdio
    time_after = float(time() * 1000)
    latency = "%.2f" % (time_after - time_before)
    #print("Episode " + str(episode) + " Latencia: " + str(latency) + " ms")

    # Decaimento de epsilon - Exploitation
    epsilon = epsilon * epsilon_decay

    total_epochs += epochs
    total_rewards.append(rewards_by_episode)
    #clear_output(wait=True)
    #print("Rewards by episodes: {}".format(rewards_by_episode))


print("\nSuccess rate epochs by episodes: {}".format(total_epochs / episodes))

# Plota rewards x  episodes

In [ ]:
# Plot total_rewards
sns.set_theme()
plt.clf()
plt.figure(figsize=(15, 6))
plt.plot(total_rewards, marker='.', linestyle='none')
plt.title('Q-learning Agente')
plt.ylabel('Rewards')
plt.xlabel('Episodes')
plt.show()

# Fase de teste

In [ ]:
# Após treino
# Cria novo ambiente e vai para um estado aleatório
state = env.reset()
# Seleciona a ação indexada máximo valor do vetor da tabela Q_table
action = np.argmax(Q_table[state])
for i in range(100):
    sleep(0.2)
    # Define o novo estado em função da ultima ação escolhida
    next_state, reward, done, info = env.step(action)
    # Seleciona a ação indexada máximo valor do vetor Q_table
    action = np.argmax(Q_table[next_state])
    clear_output(wait=True)
    env.render()
    if reward >= 20:
        print("\n\nNOVO AMBIENTE\n\n")
        sleep(0.5)
        reward = 0
        # Cria novo ambiente e vai para um estado aleatório
        state = env.reset()
        # Seleciona a ação indexada máximo valor do vetor da tabela Q_table
        action = np.argmax(Q_table[state])